In [1]:
import subprocess
import sys
import numpy as np
import os
import pandas as pd
import datetime as dt
from datetime import datetime, date, timedelta
subprocess.check_call([sys.executable,"-m","pip", "install","psycopg2-binary"])
subprocess.check_call([sys.executable,"-m","pip", "install","pandasql"])
import psycopg2

  Using cached psycopg2_binary-2.9.10-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
Using cached psycopg2_binary-2.9.10-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Using cached pandasql-0.7.3-py3-none-any.whl


In [2]:
curr_dir = os.path.abspath(os.getcwd())
data_dir = os.path.join(curr_dir,'Data')
os.makedirs(data_dir, exist_ok=True)

In [8]:
def fetch_connection_obj(psycopg2):
    return psycopg2.connect(dbname='dev',
                             host='glp-payg-database.cedhxs8itccy.us-west-2.redshift.amazonaws.com',
                             port='5439',user='developer',password='SunKing2022')

def import_psycopg2():
    try:
        import psycopg2
    except Exception as ex:
        print("psycopg2 import failed, installing psycopg2 conda package")
        subprocess.check_call([sys.executable,"-m","conda","install","psycopg2"])
        import psycopg2
    return psycopg2

def connect_to_redshift():
    try:
        con = fetch_connection_obj(psycopg2)
    except Exception as ex:
        print("fetching connection object failed, trying to import psycopg2")
        #import psycopg2
        psycopg2 = import_psycopg2()
        con = fetch_connection_obj(psycopg2)
        
    return con
    
    
def load_dataframe(data_dir, sql, csv_file_name, csv_file_prefix):
    
    csv_file_path = os.path.join(data_dir,csv_file_name)
    
    if not os.path.exists(csv_file_path):
        try:
            df = pd.read_sql(sql, con)
        except Exception as ex:
            print("read_sql failed, trying to connect to redshift")
            con = connect_to_redshift()
            df = pd.read_sql(sql, con)
        df.to_csv(os.path.join(data_dir, csv_file_prefix+"_{}.csv".format(dt.date.today())), index = False)
    else:
        df = pd.read_csv(csv_file_path, low_memory=False)
        
    if 'Unnamed: 0' in df.columns:
        df.drop(['Unnamed: 0'], axis=1, inplace=True)
        
    return df

## CBR inputs

In [9]:
sql_acc_data = """select country, to_char(registration_date_utc, 'YYYY-MM') as reg_month, product_group, primary_product, area_derived, count(*) as num_accounts 
from easybuy.accounts_final_pdt as accounts 
where has_payment_plan = true and NOT(LOWER(group_name) LIKE '%cross%') AND (NOT (LOWER(accounts.group_name) LIKE '%cross%') OR (LOWER(accounts.group_name) LIKE '%cross%') IS NULL) AND (NOT (LEFT (accounts.organization,  21)  <> 'Greenlight Planet DTV' AND  accounts.organization  <> 'Global Community Standards' AND LEFT (accounts.organization, 7)   <> 'EasyBuy') OR (LEFT (accounts.organization,  21)  <> 'Greenlight Planet DTV' AND  accounts.organization <> 'Global Community Standards' AND LEFT (accounts.organization, 7)   <> 'EasyBuy') IS NULL)  Group By 1,2,3,4,5"""

In [11]:
df_acc_data = load_dataframe(data_dir, sql_acc_data, 'df_acc_raw', 'df_acc_raw')

read_sql failed, trying to connect to redshift
fetching connection object failed, trying to import psycopg2


/tmp/ipykernel_146/1975651296.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)


In [12]:
df_acc_data

,country,reg_month,product_group,primary_product,area_derived,num_accounts
0,Kenya,2016-03,SHS Entry-Level,Sun King Home 60 EasyBuy,Kakamega,45
1,Kenya,2016-04,SHS Entry-Level,Sun King Home 60 EasyBuy,Kakamega,189
2,Kenya,2016-05,Lanterns,Sun King Pro EasyBuy,Kakamega,190
3,Kenya,2016-09,Lanterns,Sun King Pro EasyBuy,Kisumu,182
4,Kenya,2016-09,SHS Entry-Level,Sun King Home 60 EasyBuy,Kitale,59
...,...,...,...,...,...,...
233409,Kenya,2023-01,SHS with TV Upgrade,Sun King Home 400 Easybuy GSM,Matuu,1
233410,Nigeria,2023-01,SHS with TV Upgrade,Sun King Home 600 EasyBuy,Igbokoda,1
233411,Uganda,2023-01,SHS with TV Upgrade,Sun King Home 600 Mini EasyBuy,Sironko,1
233412,Uganda,2023-01,SHS with TV Upgrade,Sun King Home 600 Mini EasyBuy,Nebbi,1


In [4]:
sql_training_KPIs = """SELECT * FROM redshift_consumer_pdt.cbr_cohort3_kpis"""

In [5]:
df_KPI = load_dataframe(data_dir, sql_training_KPIs, 'df_KPI_raw', 'df_KPI_raw')

read_sql failed, trying to connect to redshift
fetching connection object failed, trying to import psycopg2


/tmp/ipykernel_3405/1010888562.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)


In [7]:
df_KPI_raw = pd.read_csv(os.path.join(data_dir,'df_KPI_raw_2025-02-19.csv'))

In [8]:
df_KPI_raw.columns

Index(['registration_date_month', 'country', 'product_group', 'area',
       'unit_age_days', 'primary_product', 'follow_on_revenue_realization',
       'at_risk_rate', 'average_cumulative_days_disabled',
       'disabled_greater_than_two_week_rate', 'count_units', 'unlocked_rate',
       'disabled_rate', 'repayment_speed_2', 'total_upfront_price_usd',
       'average_planned_repayment_days'],
      dtype='object')

In [9]:
df_KPI_raw

,registration_date_month,country,product_group,area,unit_age_days,primary_product,follow_on_revenue_realization,at_risk_rate,average_cumulative_days_disabled,disabled_greater_than_two_week_rate,count_units,unlocked_rate,disabled_rate,repayment_speed_2,total_upfront_price_usd,average_planned_repayment_days
0,2021-11,Kenya,Phones,Oyugis,90,Infinix HOT 10i,0.019980,1.000000,54.0,1.000000,1,0.000000,1.000000,0.0876,31.252311,371.0
1,2017-08,Myanmar (Burma),Lanterns,Pathein,60,Sun King Pro EasyBuy,0.204117,0.000000,6.0,0.000000,6,0.000000,0.166667,0.9939,28.560000,238.0
2,2022-01,Kenya,SHS Entry-Level Upgrade,Lodwar,360,Sun King Home 120 EasyBuy,0.580403,0.000000,97.0,0.333333,3,0.000000,0.666667,0.5984,15.872664,371.0
3,2018-10,Kenya,SHS Entry-Level,Kawangware,180,Sun King Home 60 EasyBuy,0.400187,0.000000,44.0,0.117647,51,0.000000,0.411765,0.6315,958.852428,280.0
4,2021-09,Kenya,SHS Entry-Level,Port Victoria,90,Sun King Home 60 EasyBuy,0.186438,0.043478,20.0,0.043478,23,0.000000,0.478261,0.7075,313.207008,322.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621137,2018-12,Kenya,Lanterns,Mumias,540,Sun King Pro EasyBuy,0.920408,0.000000,112.0,0.100917,218,0.756881,0.188073,0.9204,853.284046,280.0
621138,2021-10,Nigeria,NaN,Kano2,30,Sun King Home 600 EasyBuy,0.057692,0.000000,0.0,0.000000,1,0.000000,0.000000,1.3122,73.001833,378.0
621139,2020-09,Tanzania,SHS Entry-Level,Geita,180,Sun King Home 60 EasyBuy,0.306907,0.000000,15.0,0.500000,2,0.000000,0.500000,0.6457,17.241678,371.0
621140,2021-06,Tanzania,SHS with TV,Dumila,60,Sun King Home 400 Easybuy GSM,0.063978,0.250000,8.0,0.176471,17,0.000000,0.352941,0.6675,829.214034,560.0


In [13]:
df_KPI_raw['product_group'].value_counts()/621142

product_group
SHS Entry-Level            0.345254
Lanterns                   0.237488
SHS with TV                0.133657
SHS with TV Upgrade        0.088885
SHS Entry-Level Upgrade    0.064027
SHS without TV             0.040120
SHS with Stove             0.029542
Phones                     0.021026
Fans                       0.010754
No Group                   0.006279
SHS with Fan (w/o TV)      0.005200
Cash Loans                 0.002785
Water Pumps                0.001175
Water Tanks                0.001095
SHS without TV Upgrade     0.000467
Water Filters              0.000258
Inverter Large             0.000209
Cook Stoves                0.000161
SHS with Fan and TV        0.000064
Name: count, dtype: float64

In [18]:
def clean_data(df):
    
    #Renaming the columns
    df.rename(columns={'registration_date_month':'registration_month',
                            'average_cumulative_days_disabled':'avg_cum_days_disabled','average_planned_repayment_days':'avg_planned_repayment_days',
                            'disabled_greater_than_two_week_rate':'disabled_gt_two_week_rate','follow_on_revenue_realization':'frr','repayment_speed_2':'repayment_speed',
                            'total_upfront_price_usd':'upfront_price_usd'}, inplace=True)
    
    # Converting unit age days from int to string
    df = df.astype({"unit_age_days": str})

    # Handling missing values in product group
    df['product_group'].fillna('NA', inplace=True)

    # Creating index column for accounts group
    df['accounts_group'] = df['registration_month']+'_'+df['country']+'_'+ df['product_group']+'_'+df['area']+'_'+df['primary_product']
    
    print('Head of the cleaned data')
    print(df.head(1))
    
    return df


def pivot_data(df_accounts):
    
    df_other_features = df_accounts[df_accounts['unit_age_days']=='30'][['accounts_group','count_units',
                                                                                             'upfront_price_usd','avg_planned_repayment_days']].reset_index().drop(columns='index')


    df_pivot = pd.pivot_table(df_accounts,index=['accounts_group'],
                                columns='unit_age_days',
                                values =['frr','at_risk_rate','avg_cum_days_disabled','disabled_gt_two_week_rate','unlocked_rate',
                                        'disabled_rate','repayment_speed'])


    # Formatting column headings
    df_pivot.columns = df_pivot.columns.map('_'.join)

    df_pivot.reset_index(inplace=True)

    df_pivot = pd.merge(df_pivot,df_other_features, on = 'accounts_group',
                        how="inner")
    
    print('Sample of the pivoted data')
    print(df_pivot.sample(5))
    
    print('rearranging the columns')
    # Rearranging the column names
    df_pivot = df_pivot[['accounts_group','count_units','upfront_price_usd','avg_planned_repayment_days','frr_30','frr_60','frr_90','frr_180','frr_270','frr_360','frr_450','frr_540','frr_630','frr_720','repayment_speed_30','repayment_speed_60','repayment_speed_90','repayment_speed_180','repayment_speed_270','repayment_speed_360','repayment_speed_450','repayment_speed_540','repayment_speed_630','repayment_speed_720','avg_cum_days_disabled_30','avg_cum_days_disabled_60','avg_cum_days_disabled_90','avg_cum_days_disabled_180','avg_cum_days_disabled_270','avg_cum_days_disabled_360','avg_cum_days_disabled_450','avg_cum_days_disabled_540','avg_cum_days_disabled_630','avg_cum_days_disabled_720','at_risk_rate_30','at_risk_rate_60','at_risk_rate_90','at_risk_rate_180','at_risk_rate_270','at_risk_rate_360','at_risk_rate_450','at_risk_rate_540','at_risk_rate_630','at_risk_rate_720','disabled_gt_two_week_rate_30', 'disabled_gt_two_week_rate_60','disabled_gt_two_week_rate_90','disabled_gt_two_week_rate_180','disabled_gt_two_week_rate_270','disabled_gt_two_week_rate_360','disabled_gt_two_week_rate_450','disabled_gt_two_week_rate_540','disabled_gt_two_week_rate_630','disabled_gt_two_week_rate_720','unlocked_rate_30','unlocked_rate_60','unlocked_rate_90','unlocked_rate_180','unlocked_rate_270','unlocked_rate_360','unlocked_rate_450','unlocked_rate_540','unlocked_rate_630','unlocked_rate_720','disabled_rate_30','disabled_rate_60','disabled_rate_90','disabled_rate_180','disabled_rate_270','disabled_rate_360','disabled_rate_450','disabled_rate_540','disabled_rate_630','disabled_rate_720']]
    
   
    # Removing the records having no KPI's 
    df_pivot = df_pivot.loc[df_pivot['count_units'].notnull()]
    
    print('Sample of the pivoted data')
    print(df_pivot.sample(5))
    
    return df_pivot

In [19]:
df_KPI = clean_data(df_KPI_raw)

/tmp/ipykernel_3405/4045984166.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['product_group'].fillna('NA', inplace=True)


Head of the cleaned data
  registration_month country product_group    area unit_age_days  \
0            2021-11   Kenya        Phones  Oyugis            90   

   primary_product      frr  at_risk_rate  avg_cum_days_disabled  \
0  Infinix HOT 10i  0.01998           1.0                   54.0   

   disabled_gt_two_week_rate  count_units  unlocked_rate  disabled_rate  \
0                        1.0            1            0.0            1.0   

   repayment_speed  upfront_price_usd  avg_planned_repayment_days  \
0           0.0876          31.252311                       371.0   

                                accounts_group  
0  2021-11_Kenya_Phones_Oyugis_Infinix HOT 10i  


In [21]:
df_KPI

,registration_month,country,product_group,area,unit_age_days,primary_product,frr,at_risk_rate,avg_cum_days_disabled,disabled_gt_two_week_rate,count_units,unlocked_rate,disabled_rate,repayment_speed,upfront_price_usd,avg_planned_repayment_days,accounts_group
0,2021-11,Kenya,Phones,Oyugis,90,Infinix HOT 10i,0.019980,1.000000,54.0,1.000000,1,0.000000,1.000000,0.0876,31.252311,371.0,2021-11_Kenya_Phones_Oyugis_Infinix HOT 10i
1,2017-08,Myanmar (Burma),Lanterns,Pathein,60,Sun King Pro EasyBuy,0.204117,0.000000,6.0,0.000000,6,0.000000,0.166667,0.9939,28.560000,238.0,2017-08_Myanmar (Burma)_Lanterns_Pathein_Sun K...
2,2022-01,Kenya,SHS Entry-Level Upgrade,Lodwar,360,Sun King Home 120 EasyBuy,0.580403,0.000000,97.0,0.333333,3,0.000000,0.666667,0.5984,15.872664,371.0,2022-01_Kenya_SHS Entry-Level Upgrade_Lodwar_S...
3,2018-10,Kenya,SHS Entry-Level,Kawangware,180,Sun King Home 60 EasyBuy,0.400187,0.000000,44.0,0.117647,51,0.000000,0.411765,0.6315,958.852428,280.0,2018-10_Kenya_SHS Entry-Level_Kawangware_Sun K...
4,2021-09,Kenya,SHS Entry-Level,Port Victoria,90,Sun King Home 60 EasyBuy,0.186438,0.043478,20.0,0.043478,23,0.000000,0.478261,0.7075,313.207008,322.0,2021-09_Kenya_SHS Entry-Level_Port Victoria_Su...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621137,2018-12,Kenya,Lanterns,Mumias,540,Sun King Pro EasyBuy,0.920408,0.000000,112.0,0.100917,218,0.756881,0.188073,0.9204,853.284046,280.0,2018-12_Kenya_Lanterns_Mumias_Sun King Pro Eas...
621138,2021-10,Nigeria,NA,Kano2,30,Sun King Home 600 EasyBuy,0.057692,0.000000,0.0,0.000000,1,0.000000,0.000000,1.3122,73.001833,378.0,2021-10_Nigeria_NA_Kano2_Sun King Home 600 Eas...
621139,2020-09,Tanzania,SHS Entry-Level,Geita,180,Sun King Home 60 EasyBuy,0.306907,0.000000,15.0,0.500000,2,0.000000,0.500000,0.6457,17.241678,371.0,2020-09_Tanzania_SHS Entry-Level_Geita_Sun Kin...
621140,2021-06,Tanzania,SHS with TV,Dumila,60,Sun King Home 400 Easybuy GSM,0.063978,0.250000,8.0,0.176471,17,0.000000,0.352941,0.6675,829.214034,560.0,2021-06_Tanzania_SHS with TV_Dumila_Sun King H...


In [27]:
df_KPI[df_KPI['accounts_group']=='2021-11_Kenya_Phones_Oyugis_Infinix HOT 10i']

,registration_month,country,product_group,area,unit_age_days,primary_product,frr,at_risk_rate,avg_cum_days_disabled,disabled_gt_two_week_rate,count_units,unlocked_rate,disabled_rate,repayment_speed,upfront_price_usd,avg_planned_repayment_days,accounts_group
0,2021-11,Kenya,Phones,Oyugis,90,Infinix HOT 10i,0.019980,1.0,54.0,1.0,1,0.0,1.0,0.0876,31.252311,371.0,2021-11_Kenya_Phones_Oyugis_Infinix HOT 10i
26525,2021-11,Kenya,Phones,Oyugis,540,Infinix HOT 10i,0.019980,1.0,54.0,1.0,1,0.0,1.0,0.0199,31.252311,371.0,2021-11_Kenya_Phones_Oyugis_Infinix HOT 10i
42767,2021-11,Kenya,Phones,Oyugis,630,Infinix HOT 10i,0.019980,1.0,54.0,1.0,1,0.0,1.0,0.0199,31.252311,371.0,2021-11_Kenya_Phones_Oyugis_Infinix HOT 10i
52658,2021-11,Kenya,Phones,Oyugis,360,Infinix HOT 10i,0.019980,1.0,54.0,1.0,1,0.0,1.0,0.0206,31.252311,371.0,2021-11_Kenya_Phones_Oyugis_Infinix HOT 10i
58571,2021-11,Kenya,Phones,Oyugis,60,Infinix HOT 10i,0.009990,1.0,1.0,0.0,1,0.0,1.0,0.0686,31.252311,371.0,2021-11_Kenya_Phones_Oyugis_Infinix HOT 10i
68538,2021-11,Kenya,Phones,Oyugis,720,Infinix HOT 10i,0.019980,NaN,54.0,0.0,1,0.0,0.0,0.0199,31.252311,371.0,2021-11_Kenya_Phones_Oyugis_Infinix HOT 10i
92185,2021-11,Kenya,Phones,Oyugis,450,Infinix HOT 10i,0.019980,1.0,54.0,1.0,1,0.0,1.0,0.0199,31.252311,371.0,2021-11_Kenya_Phones_Oyugis_Infinix HOT 10i
123420,2021-11,Kenya,Phones,Oyugis,180,Infinix HOT 10i,0.019980,1.0,54.0,1.0,1,0.0,1.0,0.0420,31.252311,371.0,2021-11_Kenya_Phones_Oyugis_Infinix HOT 10i
133284,2021-11,Kenya,Phones,Oyugis,270,Infinix HOT 10i,0.019980,1.0,54.0,1.0,1,0.0,1.0,0.0276,31.252311,371.0,2021-11_Kenya_Phones_Oyugis_Infinix HOT 10i
139168,2021-11,Kenya,Phones,Oyugis,30,Infinix HOT 10i,0.004995,1.0,0.0,1.0,1,0.0,1.0,0.0790,31.252311,371.0,2021-11_Kenya_Phones_Oyugis_Infinix HOT 10i


In [28]:
df_pivot = pivot_data(df_KPI)

Sample of the pivoted data
                                          accounts_group  at_risk_rate_180  \
23330  2020-05_Kenya_SHS Entry-Level_Kisumu_Sun King ...               0.0   
29773  2020-11_Kenya_SHS Entry-Level_Kisii_Sun King H...               0.0   
18569  2019-12_Kenya_SHS with TV Upgrade_Sotik_Sun Ki...               0.0   
18461  2019-12_Kenya_SHS with Stove_Kakamega_Sun King...               0.0   
25566  2020-07_Kenya_SHS with TV_Samburu_Sun King Hom...               0.0   

       at_risk_rate_270  at_risk_rate_30  at_risk_rate_360  at_risk_rate_450  \
23330               0.0         1.000000               0.0               0.0   
29773               0.0         0.031250               0.0               0.0   
18569               0.0         0.111111               0.0               0.0   
18461               0.0         0.000000               NaN               NaN   
25566               0.0         0.000000               0.0               0.0   

       at_risk_rate_540

In [29]:
df_pivot.columns

Index(['accounts_group', 'count_units', 'upfront_price_usd',
       'avg_planned_repayment_days', 'frr_30', 'frr_60', 'frr_90', 'frr_180',
       'frr_270', 'frr_360', 'frr_450', 'frr_540', 'frr_630', 'frr_720',
       'repayment_speed_30', 'repayment_speed_60', 'repayment_speed_90',
       'repayment_speed_180', 'repayment_speed_270', 'repayment_speed_360',
       'repayment_speed_450', 'repayment_speed_540', 'repayment_speed_630',
       'repayment_speed_720', 'avg_cum_days_disabled_30',
       'avg_cum_days_disabled_60', 'avg_cum_days_disabled_90',
       'avg_cum_days_disabled_180', 'avg_cum_days_disabled_270',
       'avg_cum_days_disabled_360', 'avg_cum_days_disabled_450',
       'avg_cum_days_disabled_540', 'avg_cum_days_disabled_630',
       'avg_cum_days_disabled_720', 'at_risk_rate_30', 'at_risk_rate_60',
       'at_risk_rate_90', 'at_risk_rate_180', 'at_risk_rate_270',
       'at_risk_rate_360', 'at_risk_rate_450', 'at_risk_rate_540',
       'at_risk_rate_630', 'at_risk_rate

In [30]:
df_pivot.to_csv(os.path.join(data_dir,'KPIs_data_predictions.csv'),index = False)

In [31]:
df_pivot

,accounts_group,count_units,upfront_price_usd,avg_planned_repayment_days,frr_30,frr_60,frr_90,frr_180,frr_270,frr_360,...,disabled_rate_30,disabled_rate_60,disabled_rate_90,disabled_rate_180,disabled_rate_270,disabled_rate_360,disabled_rate_450,disabled_rate_540,disabled_rate_630,disabled_rate_720
0,2016-01_Kenya_Lanterns_Kakamega_Sun King Pro E...,57,403.243821,77.0,0.352180,0.626566,0.785424,0.888827,0.924767,0.935544,...,0.263158,0.315789,0.368421,0.228070,0.157895,0.122807,0.105263,0.105263,0.087719,0.052632
1,2016-02_Kenya_Lanterns_Bungoma_Sun King Pro Ea...,18,127.340154,77.0,0.425714,0.686587,0.807381,0.849048,0.856984,0.856984,...,0.277778,0.388889,0.444444,0.333333,0.333333,0.333333,0.333333,0.333333,0.222222,0.055556
2,2016-02_Kenya_Lanterns_Kakamega_Sun King Pro E...,287,2030.368011,77.0,0.365668,0.618819,0.763085,0.867551,0.897489,0.917509,...,0.229965,0.341463,0.386760,0.268293,0.188153,0.163763,0.153310,0.128920,0.108014,0.062718
3,2016-03_Kenya_Lanterns_Bungoma_Sun King Pro Ea...,114,806.487642,77.0,0.341451,0.566825,0.706045,0.812436,0.847875,0.866647,...,0.307018,0.482456,0.438596,0.307018,0.263158,0.219298,0.210526,0.201754,0.131579,0.096491
4,2016-03_Kenya_Lanterns_Kakamega_Sun King Pro E...,142,1004.572326,77.0,0.358523,0.615414,0.759101,0.854278,0.878097,0.892322,...,0.225352,0.359155,0.380282,0.260563,0.225352,0.197183,0.169014,0.161972,0.133803,0.112676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62102,2022-03_Zambia_SHS Entry-Level_Kitwe_Sun King ...,20,267.696080,382.0,0.067593,0.117694,0.169428,0.304983,0.432340,0.557559,...,0.250000,0.400000,0.300000,0.300000,0.350000,0.300000,0.250000,0.250000,0.250000,0.200000
62103,2022-03_Zambia_SHS Entry-Level_Ndola_Sun King ...,13,217.503065,382.0,0.072299,0.138023,0.199496,0.359237,0.501425,0.636675,...,0.153846,0.076923,0.230769,0.307692,0.384615,0.384615,0.461538,0.307692,0.307692,0.384615
62104,2022-03_Zambia_SHS Entry-Level_Ndola_Sun King ...,24,321.235296,382.0,0.076291,0.132562,0.192270,0.360171,0.491975,0.643764,...,0.166667,0.250000,0.291667,0.375000,0.291667,0.333333,0.291667,0.333333,0.333333,0.291667
62105,2022-03_Zambia_SHS with TV_Kitwe_Sun King Home...,5,334.620110,732.0,0.040647,0.069493,0.096416,0.188724,0.275262,0.347080,...,0.200000,0.200000,0.200000,0.600000,0.400000,0.600000,0.600000,0.400000,0.800000,0.600000


## Target data

In [32]:
sql_target = """SELECT * FROM redshift_consumer_pdt.cbr_cohort3_target"""

In [33]:
df_target = load_dataframe(data_dir, sql_target, 'df_target_raw', 'df_target_raw')

read_sql failed, trying to connect to redshift
fetching connection object failed, trying to import psycopg2


/tmp/ipykernel_3405/1010888562.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)


In [34]:
df_target

,registration_date_month,country,product_group,area,unit_age_days,primary_product,follow_on_revenue_realization,total_follow_on_revenue_usd
0,2016-03,Kenya,Lanterns,Bungoma,1080,Sun King Pro EasyBuy,0.925030,2836.891482
1,2016-04,Kenya,Lanterns,Bungoma,1080,Sun King Pro EasyBuy,0.913486,1841.490962
2,2016-07,Kenya,SHS Entry-Level,Bungoma,1080,Sun King Home 60 EasyBuy,0.852976,11904.990440
3,2016-07,Kenya,Lanterns,Bungoma,1080,Sun King Pro EasyBuy,0.872046,1841.490962
4,2016-09,Kenya,SHS Entry-Level,Eldoret,1080,Sun King Home 60 EasyBuy,0.852763,4402.158879
...,...,...,...,...,...,...,...,...
62052,2021-12,Kenya,Lanterns,Port Victoria,1080,Sun King Boom EasyBuy,0.915575,20370.327356
62053,2021-12,Kenya,SHS Entry-Level,Kiambu,1080,Sun King Home 60 EasyBuy,0.820359,4116.237022
62054,2021-12,Kenya,SHS Entry-Level,Ganze,1080,Sun King Home 120 EasyBuy,0.876259,4517.238186
62055,2021-12,Kenya,Lanterns,Homa Bay,1080,Sun King Pro EasyBuy,0.905339,4309.036210


In [35]:
def clean_data_target(df):
    
    print(df.head(1))
    # Renaming the columns
    
    df = df.drop(columns = 'unit_age_days')
    df.columns = ['registration_month','country','product_group','area','primary_product','frr_3_years','total_follow_on_revenue_usd']
    
    print(df.head(1))
    # Fill missing values with NA
    df['product_group'].fillna('NA', inplace=True)

    # Creating index column for accounts group
    df['accounts_group'] = df['registration_month']+'_'+df['country']+'_'+ df['product_group']+'_'+df['area']+'_'+df['primary_product']

    #Calculating actual FR 
    df['actual_fr'] = df['frr_3_years'] * df['total_follow_on_revenue_usd']
    
    # Keeping only required columns
    df = df[['accounts_group','frr_3_years','actual_fr','total_follow_on_revenue_usd']]
    
    print("Shape of the cleaned dataset: {}".format(df.shape))
    
    print('Head of the cleaned data')
    print(df.head(1))
    
    return df

In [36]:
df_target_final = clean_data_target(df_target)

  registration_date_month country product_group     area  unit_age_days  \
0                 2016-03   Kenya      Lanterns  Bungoma           1080   

        primary_product  follow_on_revenue_realization  \
0  Sun King Pro EasyBuy                        0.92503   

   total_follow_on_revenue_usd  
0                  2836.891482  
  registration_month country product_group     area       primary_product  \
0            2016-03   Kenya      Lanterns  Bungoma  Sun King Pro EasyBuy   

   frr_3_years  total_follow_on_revenue_usd  
0      0.92503                  2836.891482  
Shape of the cleaned dataset: (62057, 4)
Head of the cleaned data
                                      accounts_group  frr_3_years  \
0  2016-03_Kenya_Lanterns_Bungoma_Sun King Pro Ea...      0.92503   

     actual_fr  total_follow_on_revenue_usd  
0  2624.209728                  2836.891482  


/tmp/ipykernel_3405/2918119134.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['product_group'].fillna('NA', inplace=True)


In [37]:
df_target_final

,accounts_group,frr_3_years,actual_fr,total_follow_on_revenue_usd
0,2016-03_Kenya_Lanterns_Bungoma_Sun King Pro Ea...,0.925030,2624.209728,2836.891482
1,2016-04_Kenya_Lanterns_Bungoma_Sun King Pro Ea...,0.913486,1682.176213,1841.490962
2,2016-07_Kenya_SHS Entry-Level_Bungoma_Sun King...,0.852976,10154.671126,11904.990440
3,2016-07_Kenya_Lanterns_Bungoma_Sun King Pro Ea...,0.872046,1605.864827,1841.490962
4,2016-09_Kenya_SHS Entry-Level_Eldoret_Sun King...,0.852763,3753.998212,4402.158879
...,...,...,...,...
62052,2021-12_Kenya_Lanterns_Port Victoria_Sun King ...,0.915575,18650.562469,20370.327356
62053,2021-12_Kenya_SHS Entry-Level_Kiambu_Sun King ...,0.820359,3376.792087,4116.237022
62054,2021-12_Kenya_SHS Entry-Level_Ganze_Sun King H...,0.876259,3958.270616,4517.238186
62055,2021-12_Kenya_Lanterns_Homa Bay_Sun King Pro E...,0.905339,3901.138533,4309.036210


## Merge KPIs and target data

In [38]:
df_final = pd.merge(df_pivot, df_target_final, on='accounts_group', how='inner')

In [39]:
df_final

,accounts_group,count_units,upfront_price_usd,avg_planned_repayment_days,frr_30,frr_60,frr_90,frr_180,frr_270,frr_360,...,disabled_rate_180,disabled_rate_270,disabled_rate_360,disabled_rate_450,disabled_rate_540,disabled_rate_630,disabled_rate_720,frr_3_years,actual_fr,total_follow_on_revenue_usd
0,2016-01_Kenya_Lanterns_Kakamega_Sun King Pro E...,57,403.243821,77.0,0.352180,0.626566,0.785424,0.888827,0.924767,0.935544,...,0.228070,0.157895,0.122807,0.105263,0.105263,0.087719,0.052632,0.977148,1386.031419,1418.445741
1,2016-02_Kenya_Lanterns_Bungoma_Sun King Pro Ea...,18,127.340154,77.0,0.425714,0.686587,0.807381,0.849048,0.856984,0.856984,...,0.333333,0.333333,0.333333,0.333333,0.333333,0.222222,0.055556,0.903810,404.843825,447.930234
2,2016-02_Kenya_Lanterns_Kakamega_Sun King Pro E...,287,2030.368011,77.0,0.365668,0.618819,0.763085,0.867551,0.897489,0.917509,...,0.268293,0.188153,0.163763,0.153310,0.128920,0.108014,0.062718,0.958262,6843.905988,7141.998731
3,2016-03_Kenya_Lanterns_Bungoma_Sun King Pro Ea...,114,806.487642,77.0,0.341451,0.566825,0.706045,0.812436,0.847875,0.866647,...,0.307018,0.263158,0.219298,0.210526,0.201754,0.131579,0.096491,0.925030,2624.209728,2836.891482
4,2016-03_Kenya_Lanterns_Kakamega_Sun King Pro E...,142,1004.572326,77.0,0.358523,0.615414,0.759101,0.854278,0.878097,0.892322,...,0.260563,0.225352,0.197183,0.169014,0.161972,0.133803,0.112676,0.945294,3340.358794,3533.671846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62043,2022-03_Zambia_SHS Entry-Level_Kitwe_Sun King ...,20,267.696080,382.0,0.067593,0.117694,0.169428,0.304983,0.432340,0.557559,...,0.300000,0.350000,0.300000,0.250000,0.250000,0.250000,0.200000,0.754581,2374.743498,3147.102164
62044,2022-03_Zambia_SHS Entry-Level_Ndola_Sun King ...,13,217.503065,382.0,0.072299,0.138023,0.199496,0.359237,0.501425,0.636675,...,0.307692,0.384615,0.384615,0.461538,0.307692,0.307692,0.384615,0.901852,2295.020318,2544.785971
62045,2022-03_Zambia_SHS Entry-Level_Ndola_Sun King ...,24,321.235296,382.0,0.076291,0.132562,0.192270,0.360171,0.491975,0.643764,...,0.375000,0.291667,0.333333,0.291667,0.333333,0.333333,0.291667,0.873064,3036.844930,3478.376076
62046,2022-03_Zambia_SHS with TV_Kitwe_Sun King Home...,5,334.620110,732.0,0.040647,0.069493,0.096416,0.188724,0.275262,0.347080,...,0.600000,0.400000,0.600000,0.600000,0.400000,0.800000,0.600000,0.775350,4946.802929,6380.090190


In [40]:
df_final.to_csv(os.path.join(data_dir, 'KPIs_data_modelling_{}.csv'.format(dt.date.today())), index=False)

In [41]:
# df_final = pd.read_csv(os.path.join(data_dir,'KPIs_data_modelling_2024-03-15.csv'))

In [42]:
df_final.columns

Index(['accounts_group', 'count_units', 'upfront_price_usd',
       'avg_planned_repayment_days', 'frr_30', 'frr_60', 'frr_90', 'frr_180',
       'frr_270', 'frr_360', 'frr_450', 'frr_540', 'frr_630', 'frr_720',
       'repayment_speed_30', 'repayment_speed_60', 'repayment_speed_90',
       'repayment_speed_180', 'repayment_speed_270', 'repayment_speed_360',
       'repayment_speed_450', 'repayment_speed_540', 'repayment_speed_630',
       'repayment_speed_720', 'avg_cum_days_disabled_30',
       'avg_cum_days_disabled_60', 'avg_cum_days_disabled_90',
       'avg_cum_days_disabled_180', 'avg_cum_days_disabled_270',
       'avg_cum_days_disabled_360', 'avg_cum_days_disabled_450',
       'avg_cum_days_disabled_540', 'avg_cum_days_disabled_630',
       'avg_cum_days_disabled_720', 'at_risk_rate_30', 'at_risk_rate_60',
       'at_risk_rate_90', 'at_risk_rate_180', 'at_risk_rate_270',
       'at_risk_rate_360', 'at_risk_rate_450', 'at_risk_rate_540',
       'at_risk_rate_630', 'at_risk_rate

## Rough

In [5]:
sql_acc_data = """SELECT
    (TO_CHAR(DATE_TRUNC('month', accounts.registration_date_utc::DATETIME ), 'YYYY-MM')) AS "accounts.registration_date_month",
    accounts.country  AS "accounts.country",
    round(COALESCE(SUM(( portfolio.follow_on_cumulative_paid / COALESCE(accounts.current_forex,accounts.previous_forex,(COALESCE(1.0 * accounts.monthly_forex,1/accounts.exchange_rate_usd)))  ) ), 0) / NULLIF(COALESCE(SUM(accounts.follow_on_price   / COALESCE( accounts.current_forex , accounts.previous_forex  ,( COALESCE(1.0 * accounts.monthly_forex,1/accounts.exchange_rate_usd)  )) ), 0), 0), 6) AS "portfolio.follow_on_revenue_realization",
    COALESCE(CAST( ( SUM(DISTINCT (CAST(FLOOR(COALESCE( ( accounts.follow_on_price / (COALESCE(1.0 * accounts.monthly_forex,1/accounts.exchange_rate_usd)) ) ,0)*(1000000*1.0)) AS DECIMAL(38,0))) + CAST(STRTOL(LEFT(MD5(CAST( accounts.angaza_id   AS VARCHAR)),15),16) AS DECIMAL(38,0))* 1.0e8 + CAST(STRTOL(RIGHT(MD5(CAST( accounts.angaza_id   AS VARCHAR)),15),16) AS DECIMAL(38,0)) ) - SUM(DISTINCT CAST(STRTOL(LEFT(MD5(CAST( accounts.angaza_id   AS VARCHAR)),15),16) AS DECIMAL(38,0))* 1.0e8 + CAST(STRTOL(RIGHT(MD5(CAST( accounts.angaza_id   AS VARCHAR)),15),16) AS DECIMAL(38,0))) )  AS DOUBLE PRECISION) / CAST((1000000*1.0) AS DOUBLE PRECISION), 0) AS "accounts.total_follow_on_revenue_usd"
FROM easybuy_snapshot.portfolio  AS portfolio
LEFT JOIN easybuy.accounts_final_pdt -- easybuy.accounts_pdt
   AS accounts ON portfolio.account_angaza_id = accounts.angaza_id
WHERE (portfolio.unit_age_days ) = 1080 AND ((accounts.country ) IN ('Kenya', 'Nigeria', 'Tanzania', 'Uganda') AND (accounts.has_payment_plan )) AND ((DATEDIFF(days, (accounts.registration_date_utc::DATETIME), CURRENT_DATE) ) >= 1080 AND (NOT (LOWER(accounts.group_name) LIKE '%cross%' ) OR (LOWER(accounts.group_name) LIKE '%cross%' ) IS NULL) AND ((NOT (LEFT    (accounts.organization,  21)  <> 'Greenlight Planet DTV'
            AND      accounts.organization        <> 'Global Community Standards'
            AND LEFT (accounts.organization, 7)   <> 'EasyBuy'
            AND accounts.organization <> accounts.country
            ) OR (LEFT    (accounts.organization,  21)  <> 'Greenlight Planet DTV'
            AND      accounts.organization        <> 'Global Community Standards'
            AND LEFT (accounts.organization, 7)   <> 'EasyBuy'
            AND accounts.organization <> accounts.country
            ) IS NULL) AND (((((( accounts.registration_date_utc::DATETIME  ))) >= (TIMESTAMP '2020-09-01') AND ((( accounts.registration_date_utc::DATETIME  ))) < (TIMESTAMP '2021-03-01'))))))
GROUP BY
    (DATE_TRUNC('month', accounts.registration_date_utc::DATETIME )),
    2
ORDER BY
    1"""

In [6]:
df_acc_data = load_dataframe(data_dir, sql_acc_data, 'df_revenue_country', 'df_revenue_country')

read_sql failed, trying to connect to redshift
fetching connection object failed, trying to import psycopg2


/tmp/ipykernel_105/1010888562.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con)


In [7]:
df_acc_data

,accounts.registration_date_month,accounts.country,portfolio.follow_on_revenue_realization,accounts.total_follow_on_revenue_usd
0,2020-09,Kenya,0.805948,8.913338e+06
1,2020-09,Tanzania,0.810746,8.892413e+05
2,2020-09,Uganda,0.654345,4.672647e+05
3,2020-09,Nigeria,0.966497,7.628742e+05
4,2020-10,Nigeria,0.969301,8.252462e+05
5,2020-10,Tanzania,0.819895,8.540818e+05
6,2020-10,Kenya,0.812071,8.265627e+06
7,2020-10,Uganda,0.648164,4.391272e+05
8,2020-11,Tanzania,0.818182,9.573475e+05
9,2020-11,Nigeria,0.972531,9.914066e+05
